In [50]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from data_processing import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from data_transformar import *
import lightgbm as lgb
from sklearn.decomposition import PCA
from catboost import CatBoostRegressor, Pool

warnings.simplefilter("ignore")
%matplotlib inline
%load_ext autoreload
%autoreload 2

sns.set(style="darkgrid")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
from sklearn.metrics import mean_squared_error

cheat = pd.read_csv("./result-with-best.csv")

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cheat_score(model, val):
    print("RMSLE sub: " + str(rmsle(model.predict(val), np.log1p(cheat["SalePrice"]))))

def to_categorical(X):
    for c in X.columns:
        col_type = X[c].dtype
        if col_type == 'object' or col_type.name == 'category':
            X[c] = X[c].astype('category')

def evaluate(model, X, y):
    preds = model.predict(X)
    print("RMSLE: " + str(rmsle(preds, y)))

In [52]:
data = pd.read_csv("./train.csv")
data = data.drop(columns=["Id"])

data = remove_outliers(data)

y = np.log1p(data["SalePrice"])
X = data.drop(columns=["SalePrice"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=98987)
X_train_orig = X_train.copy()

transformer = DataTransformer(StandardScaler())

X_train = transformer.prepare(X_train)
X_test = transformer.prepare(X_test)

transformer.fit(X_train)

X_train = transformer.transform(X_train, False)
X_test = transformer.transform(X_test, False)

Int64Index([249, 313, 335, 523, 706, 1298], dtype='int64')


In [53]:
cat_features = np.where(X_train.loc[:, X_train.columns.values].dtypes == "object")[0]
X_train.iloc[cat_features]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1410,60.0,RL,79.000000,12420.0,Pave,None,Reg,Lvl,Inside,Gtl,...,0.0,0.0,None,None,None,0.0,6.0,2009.0,WD,Normal
723,50.0,RL,60.000000,8172.0,Pave,None,Reg,Lvl,Inside,Gtl,...,156.0,0.0,None,None,None,0.0,5.0,2008.0,WD,Normal
1371,80.0,RL,80.000000,9600.0,Pave,None,Reg,Lvl,Inside,Gtl,...,178.0,0.0,None,MnPrv,None,0.0,10.0,2008.0,WD,Normal
1440,70.0,RL,79.000000,11526.0,Pave,None,IR1,Bnk,Inside,Mod,...,0.0,0.0,None,None,None,0.0,9.0,2008.0,WD,Normal
383,45.0,RH,60.000000,9000.0,Pave,None,Reg,Lvl,Corner,Gtl,...,0.0,0.0,None,None,None,0.0,10.0,2009.0,WD,Normal
360,85.0,RL,50.289446,7540.0,Pave,None,IR1,Lvl,CulDSac,Gtl,...,192.0,0.0,None,MnPrv,None,0.0,6.0,2007.0,WD,Normal
219,120.0,RL,43.000000,3010.0,Pave,None,Reg,Lvl,Inside,Gtl,...,0.0,0.0,None,None,None,0.0,3.0,2006.0,New,Partial
513,20.0,RL,71.000000,9187.0,Pave,None,Reg,Bnk,Corner,Gtl,...,0.0,0.0,None,None,None,0.0,6.0,2007.0,WD,Normal
1170,80.0,RL,76.000000,9880.0,Pave,None,Reg,Lvl,Inside,Gtl,...,0.0,576.0,Gd,GdPrv,None,0.0,7.0,2008.0,WD,Normal
1227,20.0,RL,72.000000,8872.0,Pave,None,Reg,Lvl,Corner,Gtl,...,0.0,0.0,None,None,None,0.0,12.0,2008.0,WD,Normal


In [54]:
train_pool = Pool(X_train, y_train, cat_features=cat_features)
dev_pool = Pool(X_test, y_test, cat_features=cat_features)

cat_model = CatBoostRegressor(task_type="GPU",  devices='0:1')
cat_model.fit(train_pool, eval_set=dev_pool, verbose=0, plot=False)

evaluate(cat_model, X_train, y_train)
evaluate(cat_model, X_test, y_test)

RMSLE: 0.08410949937933657
RMSLE: 0.11873846574131477


In [55]:
data = pd.read_csv("./train.csv")
data = data.drop(columns=["Id"])

#data = remove_outliers(data)

y = np.log1p(data["SalePrice"])
X = data.drop(columns=["SalePrice"])

transformer = DataTransformer(StandardScaler())
X = transformer.prepare(X)

transformer.fit(X)

X = transformer.transform(X, False)

cat_features = np.where(X.loc[:, X.columns.values].dtypes == "object")[0]

In [58]:
train_pool = Pool(X, y, cat_features=cat_features)
#400 - 0.12927241949496218
cat_model = CatBoostRegressor(iterations=400, task_type="GPU",  devices='0:1')
cat_model.fit(train_pool, verbose=0, plot=False)

evaluate(cat_model, X, y)

RMSLE: 0.06987810052847322


In [59]:
validation = pd.read_csv("./test.csv")
val_ids = validation["Id"]
validation = validation.drop(columns=["Id"])

validation = transformer.prepare(validation)
validation = transformer.transform(validation, False)
#to_categorical(validation)

sub_predictions = cat_model.predict(validation)
print("RMSLE submission: " + str(rmsle(sub_predictions, np.log1p(cheat["SalePrice"]))))
#evaluate(lbg_model, validation,  np.log1p(cheat["SalePrice"]))

RMSLE submission: 0.12635591897893886
